In [129]:
from pysam import VariantFile

In [130]:
childVCF = VariantFile("HG002-NA24385-50x_filtered.vcf")
fatherVCF = VariantFile("HG003.hs37d5.60x.1.converted_filtered.vcf")
motherVCF = VariantFile("HG004.hs37d5.60x.1.converted_filtered.vcf")
childDeNovoVCF = VariantFile('temp.vcf', 'w', header=childVCF.header)
CHROM_POWER = 10**9

In [131]:
def writeToChildDeNovo(condition, variant):
    if condition: 
        childDeNovoVCF.write(variant)

In [132]:
def updateVariantIterator(variant, VCF, pos):
    flag = False
    if variant.chrom != "chrX": 
        variant = next(VCF, None)
        if variant.chrom != "chrX": 
            pos = (pos // CHROM_POWER + int(variant.chrom != ("chr" + str(pos // CHROM_POWER)))) * CHROM_POWER + variant.pos
    else: 
        flag = True
    return variant, pos, flag

In [133]:
flag = False
chrom = ""
pos = CHROM_POWER
while not flag:
    variant, pos , flag = updateVariantIterator(variant, fatherVCF, pos)
    if chrom != variant.chrom:
        chrom = variant.chrom
        print (str(variant.pos)+" "+ variant.chrom +" "+ str(pos) + str(flag))

10103 True 1000010103False
10286 True 2000010286False
10387 True 3000010387False
10568 True 4000010568False
10693 True 5000010693False
87296 True 6000087296False
13394 True 7000013394False
60346 True 8000060346False
10435 True 9000010435False
10651 True 10000010651False
61395 True 11000061395False
10496 True 12000010496False
16000173 True 13016000173False
16022660 True 14016022660False
17000678 True 15017000678False
10087 True 16000010087False
62171 True 17000062171False
10502 True 18000010502False
62319 True 19000062319False
61083 True 20000061083False
5102429 True 21005102429False
10510770 True 22010510770False
10112 False 22050808101False
ZAVRSIOOO


In [ ]:
def checkDeNovoOnExistingVariants(childVariant, fatherVariant, motherVariant):
    if motherVariant is None:
        return False
        #return caseSingle(childVariant, fatherVariant)#---------------------------
    if fatherVariant is None:
        return False
        #return caseSingle(childVariant, motherVariant)#------------------
    
    #both parents and the child have a variation on the same position
    
    motherGT = next(motherVariant.samples.itervalues())['GT']
    fatherGT = next(fatherVariant.samples.itervalues())['GT']
    childGT = next(childVariant.samples.itervalues())['GT']
    gt00 = (0,0)
    gt01 = (0,1)
    gt11 = (1,1)
    gt12 = (1,2)
    
    if motherGT == gt01 and fatherGT == gt01:
        #child has the same variant as one parent on one chromosome
        if childGT == gt01 and (childVariant.ref == motherVariant.ref and childVariant.alts == motherVariant.alts
                                or childVariant.ref == fatherVariant.ref and childVariant.alts == fatherVariant.alts):
            return False
        #both parents have the same (common) variant and the child got it from both parents (on both chromosomes)
        if childGT == gt11 and (childVariant.ref == motherVariant.ref and childVariant.alts == motherVariant.alts
                                and childVariant.ref == fatherVariant.ref and childVariant.alts == fatherVariant.alts): 
            return False
        #child got both parents' variants
        #-----------------------------------------special case if refs are different
        if childGT == gt12 and (childVariant.ref == motherVariant.ref and childVariant.ref == fatherVariant.ref
                                and (childVariant.alts[0] == motherVariant.alts[0] and childVariant.alts[1] == fatherVariant.alts[0]
                                     or childVariant.alts[0] == fatherVariant.alts[0] and childVariant.alts[1] == motherVariant.alts[0])): 
            return False
        return True
    if motherGT == gt01 and fatherGT == gt11:
        return False

In [21]:
def findDeNovoVariants():
    fatherVariant = next(fatherVCF.fetch())
    motherVariant = next(motherVCF.fetch())
    fatherPos = CHROM_POWER + fatherVariant.pos
    motherPos = CHROM_POWER + motherVariant.pos
    childPos = CHROM_POWER
    for childVariant in childVCF.fetch():
        
        #child variation position with child chromosome number as billions
        #to make all positions in file continual
        childPos = (childPos // CHROM_POWER + int(childVariant.chrom != ('chr' + str(childPos // CHROM_POWER)))) * CHROM_POWER + childVariant.pos
        
        #child variant pos is greater than at least one of this parents
        #iterate through parents variants until you reach child pos
        while childPos>motherPos or childPos>fatherPos:
            #if both parents have a variation on the same pos, but child doesnt
            if motherPos == fatherPos:
                #special case ----------------------------------- case 4
                fatherVariant = next(fatherVCF, None)
                motherVariant = next(motherVCF, None)
                fatherPos = (fatherPos // CHROM_POWER + int(fatherVariant.chrom != ('chr' + str(fatherPos // CHROM_POWER)))) * CHROM_POWER + fatherVariant.pos
                motherPos = (motherPos // CHROM_POWER + int(motherVariant.chrom != ('chr' + str(motherPos // CHROM_POWER)))) * CHROM_POWER + motherVariant.pos
            #both parents are smaller and one is smaller than other
            #or one parent is smaller and one parent is greater than or equal to child
            elif motherPos < fatherPos:
                #case 3
                #writeToChildDeNovo(checkHereditaryVariant(childVariant, None, MotherVariant),childVariant)---------------------
                motherVariant = next(motherVCF, None)
                motherPos = (motherPos // CHROM_POWER + int(motherVariant.chrom != ('chr' + str(motherPos // CHROM_POWER)))) * CHROM_POWER + motherVariant.pos
            else :
                #case 3
                #writeToChildDeNovo(checkHereditaryVariant(childVariant, fatherVariant, None),childVariant)---------------------
                fatherVariant = next(fatherVCF, None)
                fatherPos = (fatherPos // CHROM_POWER + int(fatherVariant.chrom != ('chr' + str(fatherPos // CHROM_POWER)))) * CHROM_POWER + fatherVariant.pos
        
        #at this point, child is <= than both parents
        
        # if child pos is smaller than both parents' pos, that's a deNovo variation
        #this variant doesnt exist in mother or father
        if childPos<motherPos and childPos<fatherPos:
            writeToChildDeNovo(True, childVariant)
            continue
         
        #at this point, child is either equal to both parents or smaller than one and equal to other
        
        if childPos<motherPos:
            #child pos is equal to father
            #father and child have a variation on the same position
            #case 2
            writeToChildDeNovo(checkDeNovoOnExistingVariants(childVariant, fatherVariant, None),childVariant)
            fatherVariant = next(fatherVCF, None)
            fatherPos = (fatherPos // CHROM_POWER + int(fatherVariant.chrom != ('chr' + str(fatherPos // CHROM_POWER)))) * CHROM_POWER + fatherVariant.pos
            continue
        
        if childPos<fatherPos:
            #child pos is equal to mother
            #mother and child have a variation on the same position
            #case 2
            writeToChildDeNovo(checkDeNovoOnExistingVariants(childVariant, None, motherVariant),childVariant)#--------
            motherVariant = next(motherVCF, None)
            motherPos = (motherPos // CHROM_POWER + int(motherVariant.chrom != ('chr' + str(motherPos // CHROM_POWER)))) * CHROM_POWER + motherVariant.pos
            continue
        
        #only option left - all three have a variation on the same position
        #case 1
        writeToChildDeNovo(checkDeNovoOnExistingVariants(childVariant, fatherVariant, motherVariant),childVariant) #---------------
        fatherVariant = next(fatherVCF, None)
        motherVariant = next(motherVCF, None)
        fatherPos = (fatherPos // CHROM_POWER + int(fatherVariant.chrom != ('chr' + str(fatherPos // CHROM_POWER)))) * CHROM_POWER + fatherVariant.pos
        motherPos = (motherPos // CHROM_POWER + int(motherVariant.chrom != ('chr' + str(motherPos // CHROM_POWER)))) * CHROM_POWER + motherVariant.pos

In [14]:
findDeNovoVariants()

10492
10904
10927
10942
13079
13116
13417
13980
14115
14590
16534
18849
20227
21796
28563
28693
30548
95580
97953
98378
98753
98921
99180
99497
101038
101158
101159
101268
101895
102334
102345
102664
103210
103444
103707
104018
104160
105700
105867
107542
108297
108775
108893
110703
110752
111371
112531
113969
121009
125271
127495
127573
128798
135040
136644
136645
136751
136817
136962
137336
150462
175224
180958
183662
184413
186835
187302
189829
191910
191944
192320
194172
194194
194262
194833
194961
195012
195142
206371
206425
207108
207150
260817
261454
261463
261478
261495
261753
266093
268130
269731
270396
271618
274015
275583
275616
277064
285882
286158
286747
286985
297077
297708
350665
350714
350724
350727
350745
368750
368752
368770
369287
369522
370001
370087
370115
370191
370244
370252
381621
382173
382231
382254
414228
414574
414665
417964
455459
463252
464057
503136
504793
505074
505462
506714
511511
511719
512032
517199
589268
589349
591413
591460
591478
591590
591602
59

2838415
2838732
2840944
2844810
2849663
2852326
2852394
2859517
2859549
2860196
2863249
2864431
2865645
2866028
2879990
2894453
2921009
2921013
2921063
2921065
2921146
2921159
2921186
2921206
2921227
2921235
2921271
2921277
2921285
2921359
2921360
2921397
2921432
2921527
2921528
2921533
2921569
2921803
2921896
2921906
2921948
2922141
2922151
2922233
2922443
2922940
2923192
2923196
2923260
2923787
2923800
2924045
2924136
2924322
2924348
2924360
2924626
2924766
2924892
2925148
2925149
2925315
2925430
2925705
2925754
2925916
2925918
2926182
2926499
2926615
2926657
2926681
2926746
2926781
2926789
2926812
2926816
2926863
2926912
2927285
2927502
2927689
2927792
2927809
2927836
2928397
2928409
2929076
2929187
2929279
2929709
2930341
2930923
2931432
2932625
2932923
2932992
2933202
2933547
2933616
2934673
2935641
2936260
2936438
2937194
2937640
2937708
2938503
2941983
2943159
2944874
2945010
2945066
2945275
2945317
2945755
2945915
2946290
2946550
2947398
2947450
2949722
2949724
2949955
2950367


5161159
5170686
5173931
5181251
5182121
5184701
5190845
5212151
5214526
5214588
5214618
5214626
5214684
5214688
5214771
5215337
5215661
5216716
5216718
5217377
5218575
5219286
5219562
5222491
5222757
5224016
5225393
5226653
5227048
5227099
5227622
5227849
5228015
5228316
5230155
5234308
5235116
5237785
5238707
5241944
5246172
5246237
5251992
5252306
5252630
5252729
5252801
5252929
5252965
5253385
5253602
5253850
5254677
5254769
5255261
5255284
5255351
5255421
5255718
5255843
5256335
5256544
5256778
5257083
5258011
5258122
5282235
5302232
5303133
5303134
5303138
5303837
5303941
5303999
5306453
5307372
5307565
5307834
5308047
5308077
5308981
5312969
5314821
5314880
5315284
5316414
5317106
5318225
5319564
5320570
5321999
5323349
5323547
5324883
5332124
5332785
5332805
5333194
5336284
5336690
5340485
5341304
5341854
5342810
5343525
5345042
5345052
5364046
5383858
5384064
5386831
5389092
5391417
5402858
5403877
5404272
5404929
5406466
5410891
5411457
5413748
5414019
5414024
5414152
5415218


7905428
7908032
7908808
7908954
7909249
7909546
7914221
7914236
7914326
7918175
7929967
7953147
7978392
7979889
7990498
7990877
7993593
8007442
8007499
8007505
8007507
8007774
8063002
8089256
8089267
8105032
8128063
8128256
8128563
8128937
8129149
8129184
8129213
8129794
8130918
8131144
8131732
8132035
8132865
8139446
8140518
8140990
8141067
8156926
8158146
8160005
8161713
8165655
8183745
8213491
8230871
8233040
8239979
8255459
8275528
8281091
8299065
8299717
8306954
8311020
8322034
8322035
8322255
8322259
8322418
8323214
8332335
8349612
8358584
8361032
8361143
8365721
8371547
8375632
8376742
8385993
8386024
8387177
8397588
8462493
8462850
8466082
8469081
8469464
8478243
8479209
8480130
8482219
8482640
8486755
8496485
8496893
8497260
8498715
8498732
8500586
8501082
8501144
8501187
8501194
8501200
8501227
8501271
8501472
8501777
8501827
8502034
8502074
8502198
8502509
8509812
8512949
8513412
8519242
8519390
8520752
8521168
8521194
8523661
8529287
8531007
8534908
8535835
8536123
8536834


11683514
11684161
11684740
11684928
11689292
11689900
11690455
11690929
11693757
11695611
11695613
11695868
11697042
11699835
11702619
11704224
11705749
11708526
11717661
11720593
11720613
11721711
11722724
11722829
11723394
11727635
11727667
11727668
11728818
11729726
11730314
11730405
11730630
11730997
11731111
11731242
11731850
11733609
11733617
11734496
11736111
11738678
11741030
11743428
11743639
11744533
11745326
11746794
11749104
11749678
11751226
11751978
11756616
11757047
11757893
11759539
11760835
11764076
11767775
11770123
11770810
11770994
11771246
11771297
11771486
11771558
11772043
11772864
11772894
11772904
11773038
11773853
11773919
11774708
11775032
11775499
11775878
11776126
11776261
11776384
11776625
11777416
11777524
11777592
11777625
11777645
11777954
11778394
11778459
11778639
11778705
11778784
11778791
11778977
11779144
11779169
11779378
11779866
11779909
11781067
11781864
11781888
11782087
11783110
11783862
11784157
11784770
11785112
11785461
11786035
11786937
1

...

239277142
239277390
239277391
239277534
239278356
239278398
239278400
239278402
239278404
239278426
239278428
239278502
239278512
239278640
239278861
239281624
239281664
239283432
239284207
239285247
239287532
239291192
239291198
239300790
239300816
239302241
239302625
239303477
239303599
239304307
239307048
239307805
239307808
239308384
239308746
239308777
239309739
239310142
239310381
239310714
239311699
239312235
239312440
239312461
239312841
239313069
239313194
239313369
239313553
239318091
239323716
239328168
239333169
239373859
239384993
239396331
239404408
239407417
239407419
239408064
239408503
239409595
239410370
239411720
239412951
239415418
239418117
239422576
239422664
239422804
239422810
239423559
239424212
239426241
239426697
239426923
239428860
239429783
239429973
239430543
239431836
239432212
239432977
239433865
239435240
239435457
239435569
239435848
239437433
239438247
239438540
239439809
239443558
239445037
239445942
239446137
239446142
239448207
239449866
239451562


241850290
241850771
241850836
241852518
241853536
241853707
241854429
241854524
241854681
241854737
241855125
241855281
241855677
241855678
241855925
241855940
241855947
241856263
241856278
241856529
241856697
241857089
241857090
241857193
241859394
241860288
241861796
241861942
241862426
241863042
241863423
241863543
241863743
241863834
241864725
241865054
241865058
241865177
241865195
241865502
241865799
241865904
241865947
241866029
241866849
241867181
241867303
241868374
241868537
241869736
241869764
241869900
241871911
241871971
241872641
241873078
241873203
241873808
241874702
241874834
241875734
241885854
241885859
241885917
241888336
241888409
241888649
241888964
241889085
241891967
241892848
241898052
241898923
241899468
241899860
241900772
241901204
241901730
241903099
241903210
241903647
241912751
241917957
241918538
241918564
241918633
241918678
241918910
241919594
241920080
241920504
241921247
241921391
241921800
241921839
241922051
241922342
241922425
241922465
241922571


244373332
244376453
244381516
244382216
244382417
244385745
244386694
244390205
244390492
244392659
244392770
244394245
244394248
244394591
244396796
244397817
244400920
244403665
244405699
244415077
244423764
244429989
244432332
244432660
244434624
244435553
244439426
244439441
244439654
244446669
244455224
244470120
244489450
244490254
244491567
244492050
244497245
244498175
244500508
244501861
244505966
244507902
244508831
244508985
244510835
244513710
244514828
244515959
244516105
244516174
244516191
244516207
244516335
244516402
244516509
244516605
244516761
244517530
244517676
244517773
244518163
244518199
244518851
244518974
244518979
244519068
244519205
244519364
244519479
244519598
244519830
244520023
244520051
244520199
244520324
244520604
244520965
244521045
244521236
244521365
244521397
244521651
244521918
244522322
244522570
244522792
244522812
244523030
244523065
244523144
244523309
244523321
244523359
244523365
244524424
244524550
244524553
244524574
244524638
244524726


246687994
246692758
246694031
246696771
246696792
246697372
246697376
246698383
246699690
246703093
246710417
246714432
246715820
246719355
246723744
246724751
246791686
246795119
246796293
246809625
246812739
246814924
246814954
246815583
246815729
246815845
246816504
246817182
246817255
246817596
246817961
246819464
246819513
246819871
246820221
246821010
246821041
246821067
246821367
246821879
246822023
246822499
246822830
246823336
246823699
246823918
246824511
246824877
246825697
246825721
246825869
246825883
246825934
246826243
246826397
246827128
246827792
246827857
246828110
246829821
246843740
246868988
246906906
246959310
247091422
247091424
247091486
247092640
247111916
247118959
247123595
247129931
247131371
247160273
247174638
247174848
247181328
247181571
247182538
247182650
247182713
247182887
247183372
247184413
247185776
247186375
247187981
247189878
247191279
247191801
247192777
247192780
247193153
247194499
247194521
247194525
247194527
247194529
247194531
247194533


248685906
248744377
248747028
248750298
248764484
248770538
248775684
248775703
248779296
248803632
248803643
248803659
248803660
248803911
248819398
248820444
248829712
248845243
248875308
248877988
248878182
248884347
248907650
248907868
248932575
248933569
248936976
248938553
248938558
248942003
248942240
248942562
248943839
248944289
248944749
248944897
248945106
248945567
248945579
248945586
248945614
248945624
248945670
248945674
248945679
248945809
248945883
248946018
248946154
248946202
248946252
248946262
248946270
248946276
10250
